In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs 
# todo NOTE: also this script will encounter runtime/space error since we used device IDs as a key while merge dataframes 

import pandas as pd
import numpy as np

file_cleaned_flow = '../../Endpoint Mapping Data/Cleaned Flow/cleaned_flow_stat.csv'
cleaned_flow = pd.read_csv(file_cleaned_flow)

cleaned_flow['super_vendor'] = cleaned_flow.apply(
    lambda row: row.vendor_name.lower() if row.vendor_name==row.vendor_name
    else row.gpt_clean_vendor,
    axis=1)


cleaned_flow['generic_category'] = cleaned_flow.apply(
    lambda row: row.man_generic_category if row.man_generic_category==row.man_generic_category
    else row.gpt_generic_category,
    axis=1)


# read all party mapping files
file_all_party_mapping = '../../Endpoint Mapping Data/Domain Data/all_party_mapping.csv'
all_party_mapping = pd.read_csv(file_all_party_mapping)
# drop extra columns 
all_party_mapping = all_party_mapping[['super_vendor', 'domain', 'party_labels']].drop_duplicates()

# marge with clean flow
clean_flow_party_label = pd.merge(cleaned_flow,
                                  all_party_mapping,
                                  on=['super_vendor', 'domain'],
                                  how='left'
                                  )


# read user information file 
file_user_device_timezone = '../../Inspector Dataset/New data/user_device_timezone.csv'
user_device_timezone = pd.read_csv(file_user_device_timezone)
user_device_timezone = user_device_timezone[['device_id', 'user_key', 'user_country', 'timezone']]

# merge with timezone file 
clean_flow_party_label = pd.merge(clean_flow_party_label,
                                  user_device_timezone,
                                  on=['device_id'],
                                  how='left'
                                  )

print(len(clean_flow_party_label['device_id'].unique()))
unique_categories = clean_flow_party_label['generic_category'].unique()
print(list(unique_categories))


unique_categories = ['Media/TV',
                     'Voice Assistant',
                     'Surveillance',
                     'Home Automation',
                     'Home Appliance ',
                     'Game Console',
                     'Generic IoT',
                     'Work Appliance',
                     'Vehicle']


### RQ 4

#### Who are the dominant support parties within the IoT ecosystem?

In [ ]:
# finding top support parties

from cleanco import basename

devices_df = clean_flow_party_label
support_flows = devices_df[devices_df['party_labels']==2]
support_parties = support_flows['domain'].drop_duplicates().reset_index(drop=True)

# load combined information
file_combined_domain2org = '../../Endpoint Mapping Data/Domain Data/First Party Mapping/domain2org_all_possible_sources.csv'
domain2org = pd.read_csv(file_combined_domain2org).rename(columns={'remote_hostname':'domain'})
# merge 
support_parties = pd.merge(support_parties,
                           domain2org,
                           on=['domain'],
                           how='left'
                           )

# find support organization
support_parties['organization'] = support_parties.apply(lambda row: row.netify_org if row.netify_org==row.netify_org
else basename(basename(row.python_whois_org).capitalize() if (row.python_whois_org==row.python_whois_org) & (len(str(row.python_whois_org).strip())>0)
              else basename(basename(row.bash_whois_org).capitalize() if (row.bash_whois_org==row.bash_whois_org) & (len(str(row.bash_whois_org).strip())>0)
                            else (row.xclusive_org.capitalize() if row.xclusive_org==row.xclusive_org
                                  else row.domain))), axis=1)

support_parties = support_parties[['domain', 'organization']]

# add support organization in the flow table
support_flows = pd.merge(support_flows, support_parties, on='domain', how='left')

# get support organization and count of supported devices 
support_organization = support_flows.groupby(['organization'])[['device_id']].nunique().sort_values(by='device_id', ascending=False).reset_index().rename(columns={'device_id':'device_count'}).set_index('organization')



## lIST TOP SUPPORT PARTIES

for org in support_organization.index:
    device_count = support_organization['device_count'].loc[org]

    if device_count < 20:
        continue
    print(org.ljust(15), end=' & \t')
    # print(org ,'&', device_count, end=' & ')

    vendor_device_count = support_flows[(support_flows['organization']==org)].groupby(['super_vendor'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    for vendor in vendor_device_count.index:
        if vendor_device_count['device_id'].loc[vendor] < 2:
            continue
        print('{0}({1}), '.format(vendor, vendor_device_count['device_id'].loc[vendor]), end='')

    print('\\\\[3pt]')
    print()


In [ ]:
## lIST OF VENDORS WITH MULTIPLE SUPPORT PARTIES
vendor_name_device_count = support_flows.groupby(['super_vendor'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)

for vendor in vendor_name_device_count.index:
    device_count = vendor_name_device_count['device_id'].loc[vendor]

    ## skip if device count is less than 3
    if device_count < 20:
        continue

    print(vendor.ljust(15), end=' & \t')

    support_device_count = support_flows[(support_flows['super_vendor']==vendor)].groupby(['organization'])[['device_id']].nunique().sort_values(by='device_id', ascending=False)
    
    for support in support_device_count.index:
        if support_device_count['device_id'].loc[support] < 5:
            continue
        print('{0}({1}), '.format(support, support_device_count['device_id'].loc[support]), end='')

    print('\\\\[2pt]')
    print()